# UNLPD Crime Analysis
The goal of this project was to analyze University of Nebraska-Lincoln Police Department crime data to identify when crime reports are abnormally high or low. To do so, I collected 13 years of data from UNLPD, taken from the [Daily Crime and Fire Log](https://scsapps.unl.edu/policereports/MainPage.aspx). This program cleans up that data and calculates the average number of occurrences for each crime in a normal month.  
Then, it gets the data for this month. If this month's counts are abnormally high or low, a message is created, which is sent to a Slack channel to notify those who are interested.

## Requirements
This program requires `pandas`, `slackclient` and `selenium`. It also requires a Slack bot API token. More information on how to set up Slack is below.

## Imports and functions
These are the imports and functions needed to run the rest of the program.

### Imports  
First, import the necessary libraries.

In [1]:
import os #To get Slack API environment variable
import pandas as pd #For most data manipulations
import subprocess #To run in2csv
from io import StringIO #To convert a string to a file
from collections import defaultdict #For counting crimes
from datetime import datetime #To check today's date
from calendar import monthrange #To find last day of month
from slackclient import SlackClient #To post to Slack
from selenium import webdriver #To scrape UNLPD's data

### clean_data()
This checks the data to make sure it's valid. 
Then, it removes the currency characters. 
It fixes some data types, and it creates a new column with the month of each crime. 
Finally, it sets the index of the dataframe to the Reported column.

In [21]:
def clean_data(df): 
    # Make sure there are no null values for the Case Number, Reported time, Location, Stolen amount and Damaged amount
    assert(df['Case #'].count() == len(df))
    assert(df['Reported'].count() == len(df))
    assert(df['Location'].count() == len(df))
    assert(df['Stolen'].count() == len(df))
    assert(df['Damaged'].count() == len(df))
    
    #Replace non-numerical characters and cast data type to float
    df['Stolen'] = df['Stolen'].str.replace(',','')
    df['Stolen'] = df['Stolen'].str.replace('$','')
    df['Stolen'] = df['Stolen'].astype(float)
    df['Damaged'] = df['Damaged'].str.replace(',','')
    df['Damaged'] = df['Damaged'].str.replace('$','')
    df['Damaged'] = df['Damaged'].astype(float)
    
    #Cast data type to datetime
    df['Reported'] = pd.to_datetime(df['Reported'])
    
    #Double-check data types
    print(df.dtypes)
    
    #Create a new column with just the year and month from the Reported column
    df['Month'] = df['Reported'].dt.to_period('M')
    
    #Set index to Reported column; allows for slicing by month
    df2 = df.set_index(['Reported'])
    
    return df2

### count_crimes()
This counts the occurrences of crimes in a given dataframe.
For each type of crime, it slices the data by month.
Then it counts how many times the crime appears in that month.
Those counts are saved to a dictionary.
Finally, the list of dictionaries is converted back to a dataframe.
Pass the all_years flag if your data contains more than one month.

In [22]:
def count_crimes(df, all_years=False):
    #This holds the dictionary for each crime
    months_count = []
    #For each crime present in the dataframe
    for crime in df['Incident Code'].unique():
        print(crime)
        crime_dict = defaultdict(list)
        crime_dict['Crime'] = crime
        #For each month in the dataframe
        for month in df['Month'].unique():
            #Slice the dataframe for one month's data
            month_subset = df[str(month)]
            #Filter the subset for instances of the crime
            crime_subset = month_subset[month_subset['Incident Code'] == crime]
            #If multiple months, save the count with the month
            if all_years:
                crime_dict[str(month)] = len(crime_subset)
            #Otherwise, just save it with "Month"
            else:
                crime_dict['Month'] = len(crime_subset)
        #Append the dictionary to the months_count list
        months_count.append(crime_dict)
    #Convert the list into another dataframe
    months_count_df = pd.DataFrame(months_count)
    #To help with speed, save it to a csv
    if all_years:
        months_count_df.to_csv('month_count.csv', index=False)
    return months_count_df

### calculate_stats()
This calculates the mean number of occurences for each crime.
Then it calculates the standard deviation.
Finally, it creates upper and lower thresholds.
Upper threshold equals mean plus 1 standard deviation.
Lower threshold equals mean minus 1 standard deviation.

In [20]:
def calculate_stats(df):
    #Creates a dataframe with the unique crimes
    std_df = df.filter(['Crime'], axis=1)
    #Adds a column with the mean count for each crime
    std_df['mean'] = df.mean(axis=1)
    #Adds a column with the standard deviation for each crime
    std_df['std'] = df.std(axis=1)
    #Adds a column with a lower threshold
    std_df['lower'] = std_df['mean'] - std_df['std']
    #Adds a column with an upper threshold
    std_df['upper'] = std_df['mean'] + std_df['std']
    #Save the data to a csv
    std_df.to_csv('std.csv', index=False)
    return std_df

### check_last_day()
This checks today's date to see if it's the last day of the month. It uses the monthrange function from the calendar library.

In [ ]:
def check_last_day():
    #Get today's date
    today = datetime.today()
    #monthrange(year, month) returns weekday of first day of the month and number of days in month, for the specified year and month.
    if today.day == monthrange(today.year, today.month)[1]:
        return True
    else:
        return False

### post_to_slack()
This sends an API call to Slack to post a message to a specified channel.

In [ ]:
def post_to_slack(message):
    #Send the API call to post a message
    slack_client.api_call("chat.postMessage", channel='C49E1F45R', text=message, as_user=True)

### find_outliers()
This checks the data to see which crimes have crossed the threshold. 
If it's not the last day of the month, it will only check crimes that have occurred this month for crossing the upper threshold. 
If it is the last day of the month, it will check all the possible crimes for crossing the upper threshold or not meeting the lower threshold.
The `flagged_crimes` list is a list of crimes that will create an alert. We chose these for our purposes, but you can add or remove crimes as you wish.
If a crime met a threshold and it's in `flagged_crimes`, it will create a message and send it to Slack using `post_to_slack()`.

In [ ]:
def find_outliers(all_years_stats, month_count):
    #This is the list of crimes we decided we were interested in
    flagged_crimes = [
        "LOST OR STOLEN ITEM",
        "FRAUD - CREDIT CARDS/ATM/BANK CARD",
        "LARCENY - FROM MOTOR VEHICLE",
        "NARCOTICS - POSSESSION",
        "BURGLARY",
        "LARCENY - FROM BUILDING",
        "ALCOHOL - DWI",
        "ALCOHOL - DRUNK",
        "ALCOHOL - MINOR IN POSSESSION",
        "VANDALISM - OTHER",
        "LARCENY - STOLEN BIKE",
        "VANDALISM - BY GRAFFITI",
        "NARCOTICS - OTHER",
        "NARCOTICS - SALE/DELIVER",
    ]
    #If it's the last day, merge the data and keep everything
    if check_last_day():
        merged = pd.merge(all_years_stats, month_count, on='Crime', how='outer')
    #Otherwise, only keep the data for crimes that have happened this month
    else:
        merged = pd.merge(all_years_stats, month_count, on='Crime', how='inner')
    #For each row, check if the count has crossed the upper bound
    for index, row in merged.iterrows():
        if row['upper'] < row['Month'] and row['Crime'] in flagged_crimes:
            #If it has happened more than once, use plural words
            if row['Month'] != 1:
                message = "This month, there have been {month_total} {crime} incidents reported. There are normally {mean} incidents reported in a month, and one standard deviation above is {upper_bound}.".format(crime=row['Crime'], upper_bound=round(row['upper'], 2), month_total=row['Month'], mean=round(row['mean'], 2))
            #Otherwise, use singular words
            else:
                message = "This month, there has been {month_total} {crime} incident reported. There are normally {mean} incidents reported in a month, and one standard deviation above is {upper_bound}.".format(crime=row['Crime'], upper_bound=round(row['upper'], 2), month_total=row['Month'], mean=round(row['mean'], 2))
            #Print the message here
            print(message)
            #Post the message to Slack
            post_to_slack(message)

## Preparing the historical data
This works with a csv of historical data. I created it by downloading all the crimes for each year and concatenating them into one file. It reads the file into a Pandas dataframe and runs the `clean_data()`, `count_crimes()` and `calculate_stats()` functions on it.

In [12]:
#Read in the csv file
all_years = pd.read_csv('original_data/all_years.csv')
#Clean the data
all_years_clean = clean_data(all_years)
#Count the crime occurences
all_years_count = count_crimes(all_years_clean, all_years=True)
#Calculate the thresholds
all_years_stats = calculate_stats(all_years_count)

Case #                    object
Incident Code             object
Reported          datetime64[ns]
Case Status               object
Start Occurred            object
End Occurred              object
Building                  object
Location                  object
Stolen                   float64
Damaged                  float64
Summary                   object
dtype: object


### To use the stats data without recalculating it:
The previous cell can take a while to run, so the next cell just uses the csv that is written during the `calculate_stats()` function.

In [5]:
#This pulls the most recent saved csv
all_years_stats = pd.read_csv('std.csv')

## Finding current totals
Once the historical data is prepared, the next step is to check this month's data against it.

### Downloading this month's data
This uses selenium to get this month's data.
It opens an instance of Google Chrome.
Then, it fills out the forms to access the data.
Finally, it downloads the csv, likely to your downloads folder.

In [6]:
#This is needed to set up selenium
path_to_chromedriver = '/Users/Aidan/Downloads/chromedriver'
browser = webdriver.Chrome(executable_path=path_to_chromedriver)
#The URL to the Daily Crime and Fire Log
url = "https://scsapps.unl.edu/policereports/MainPage.aspx"
#Go to the URL
browser.get(url)
#Find the advanced search button and click it
browser.find_element_by_id('ctl00_ContentPlaceHolder1_AdvancedSearchButton').click()
#Find the first date field, hit tab and hit '01'. 
#This sets the date to the first day of the month
browser.find_element_by_id('ctl00_ContentPlaceHolder1_DateRange_MonthText1').send_keys('\t01')
#Find the search button and click it
browser.find_element_by_id('ctl00_ContentPlaceHolder1_SearchButton').click()
#Switch to the iframe on the page
browser.switch_to.frame(browser.find_element_by_id('ctl00_ContentPlaceHolder1_ViewPort'))
#Find the export button and click it
browser.find_element_by_id('ExportButton').click()

### Converting the data to a dataframe
Once this month's data is downloaded, we need to bring it in here. Be warned, this is assuming the file has the generic name given by UNLPD. If you have another copy in the folder from earlier, your new copy will have (1) after it, causing you to not use the new file.  
Once we get the data, we need to prepare it. So, this converts it from an Excel file to a csv using `in2csv`. Since `in2csv` is run from the command line, I used subprocess to run it here. `StringIO` converts the string returned by `in2csv` into a file to read into a dataframe. The first eight rows are skipped to remove the junk at the top of the downloaded file.

In [6]:
#Runs in2csv on the downloaded file and converts it to UTF-8
csv_data = subprocess.check_output(["in2csv", "/Users/Aidan/Downloads/DailyCrimeLogSummary.xls"]).decode("utf-8")
#Creates a file instance for pandas to use on the next line
csv_file_instance = StringIO(csv_data)
#Reads in the csv to a dataframe, skipping the first eight rows
month_df = pd.read_csv(csv_file_instance, skiprows=8)

Next, we run the data through the same cleaning and counting functions as our historical data.

In [7]:
#Clean the data
month_clean = clean_data(month_df)
#Count the crime occurences
month_count = count_crimes(month_clean)

Case #                    object
Incident Code             object
Reported          datetime64[ns]
Case Status               object
Start Occurred            object
End Occurred              object
Building                  object
Location                  object
Stolen                   float64
Damaged                  float64
Summary                   object
dtype: object
LOST OR STOLEN ITEM
NARCOTICS - POSSESSION
ALCOHOL - MINOR IN POSSESSION
MISC - OTHER
ACCIDENTS - P.D. H&R NOT REPORTABLE
LARCENY - STOLEN BIKE
ALCOHOL - DWI
VANDALISM - BY GRAFFITI
ALCOHOL - DRUNK
DISTURBANCE - OTHER
SUSPICIOUS VEHICLE
MEDICAL EMERGENCY
TRESPASSING
TELEPHONE - OBSCENE CALLS
VANDALISM - OTHER
FIRE - FALSE ALARM
LARCENY - FROM BUILDING
LARCENY - SHOPLIFTING
SS - CHECK WELFARE OF PERSON
ASSAULT - DOMESTIC
NARCOTICS - SALE/DELIVER
ACCIDENTS - P.D. REPORTABLE
OUTSIDE - O.P.S. OTHER
LARCENY - OTHER OR FROM OPEN AREA
NARCOTICS - OTHER
DISTURBANCE - DOMESTIC
SEX OFFENSE - RAPE
TELEPHONE - OTHER
ASSAULT - N

## Setup Slack
This program is designed to post alerts to Slack. To do so, you need a Slack workspace with a bot user. Get the API token for the bot user and save it as the environment variable `UNLPD_BOT_TOKEN`. Then, this will set up the connection and prepare it to send alerts.

In [13]:
#Get the API token from the environment variable
bot_token = os.environ.get('UNLPD_BOT_TOKEN')
#Set up the connection to Slack using SlackClient
slack_client = SlackClient(bot_token)

## Find outliers
Finally, we compare this month's data to the historical data using `find_outliers()`. If an alert is created, it prints here, and it also sends to Slack.

In [15]:
find_outliers(all_years_stats, month_count)

This month, there have been 20 NARCOTICS - POSSESSION incidents reported. There are normally 9.5 incidents reported in a month, and one standard deviation above is 15.91.
This month, there has been 1 FRAUD - IMPERSONATION incident reported. There are normally 0.17 incidents reported in a month, and one standard deviation above is 0.59.
This month, there have been 2 DISTURBANCE - DOMESTIC incidents reported. There are normally 0.48 incidents reported in a month, and one standard deviation above is 1.15.
This month, there has been 1 SEX OFFENSE - MOLEST/FONDLING incident reported. There are normally 0.17 incidents reported in a month, and one standard deviation above is 0.59.
This month, there has been 1 LARCENY - SHOPLIFTING incident reported. There are normally 0.27 incidents reported in a month, and one standard deviation above is 0.77.
This month, there have been 4 SEX OFFENSE - RAPE incidents reported. There are normally 0.35 incidents reported in a month, and one standard deviation